# Trace-Based Retrieval
A variant of CMR/InstanceCMR that retrieves and reinstances stored traces rather than prototypical representations.

## Model

In [4]:
@jitclass(cmr_spec)
class Trace_Retrieval_CMR:

    def __init__(self, item_count, presentation_count, encoding_drift_rate,
                 start_drift_rate, recall_drift_rate, shared_support, 
                 item_support, learning_rate, primacy_scale, primacy_decay, 
                 stop_probability_scale, stop_probability_growth, 
                 choice_sensitivity):
        """
        Same as for Instance_CMR, except preallocated item probabilities vector is of length presentation_count.
        We also add self.identities to track item corresponding to each memory trace.
        """
        
        # store initial parameters
        self.item_count = item_count
        self.encoding_drift_rate = encoding_drift_rate
        self.delay_drift_rate = delay_drift_rate
        self.start_drift_rate = start_drift_rate
        self.recall_drift_rate = recall_drift_rate
        self.shared_support = shared_support
        self.item_support = item_support
        self.learning_rate = learning_rate
        self.primacy_scale = primacy_scale
        self.primacy_decay = primacy_decay
        self.stop_probability_scale = stop_probability_scale
        self.stop_probability_growth = stop_probability_growth
        self.choice_sensitivity = choice_sensitivity
        
        # at the start of the list context is initialized with a state 
        # orthogonal to the pre-experimental context
        # associated with the set of items
        self.context = np.zeros(item_count + 1)
        self.context[0] = 1
        self.preretrieval_context = self.context
        self.recall = np.zeros(item_count) # recalls has at most `item_count` entries
        self.retrieving = False
        self.recall_total = 0
        
        # predefine activation weighting vectors
        self.item_weighting = np.ones(item_count+presentation_count)
        self.context_weighting = np.ones(item_count+presentation_count)
        self.item_weighting[item_count:] = learning_rate
        self.context_weighting[item_count:] = \
            primacy_scale * np.exp(-primacy_decay * np.arange(presentation_count)) + 1
        self.all_weighting = self.item_weighting * self.context_weighting
        
        # preallocate for outcome_probabilities - one for each presentation this time!
        self.probabilities = np.zeros((presentation_count + 1))
        
        # initialize memory
        # we now conceptualize it as a pairing of two stores Mfc and Mcf respectively
        # representing feature-to-context and context-to-feature associations
        mfc = np.eye(item_count, item_count + 1, 1) * (1 - learning_rate)
        mcf = np.ones((item_count, item_count)) * shared_support
        for i in range(item_count):
            mcf[i, i] = item_support
        mcf = np.hstack((np.zeros((item_count, 1)), mcf))
        self.memory = np.zeros((item_count + presentation_count, item_count * 2 + 2))
        self.memory[:item_count,] = np.hstack((mfc, mcf))
        self.encoding_index = item_count
        self.items = np.eye(item_count, item_count + 1, 1)
        
        self.identities = np.zeros(item_count + presentation_count, dtype=int32)
        self.identities[:item_count] = np.arange(item_count, dtype=int32)
        
    def experience(self, experiences):

        for i in range(len(experiences)):
            self.memory[self.encoding_index, :self.item_count+1] = experiences[i]
            self.update_context(self.encoding_drift_rate, self.memory[self.encoding_index])
            self.memory[self.encoding_index, self.item_count+1:] = self.context
            self.encoding_index += 1
            
    def update_context(self, drift_rate, experience=None):
        """
        InstanceCMR retrieves an echo representation over memory based on the probe.
        Our version should instead reinstate the contextual state corresponding to a particular memory trace.
        Though perhaps echo-based reinstatement is appropriate in some circumstances?
        """

        # first pre-experimental or initial context is retrieved
        if experience is not None:
            context_input = self.echo(experience)[self.item_count + 1:]
            context_input = context_input / np.sqrt(np.sum(np.square(context_input))) # norm to length 1
        else:
            context_input = np.zeros((self.item_count+1))
            context_input[0] = 1

        # updated context is sum of context and input, modulated by rho to have len 1 and some drift_rate
        rho = np.sqrt(1 + np.square(drift_rate) * (np.square(self.context * context_input) - 1)) - (
                drift_rate * (self.context * context_input))
        self.context = (rho * self.context) + (drift_rate * context_input)
        
    def echo(self, probe):

        return np.dot(self.activations(probe), self.memory[:self.encoding_index])
    
    def activations(self, probe):
        """
        Retrieves an activation level for each trace in memory based on similarity to a probe
        and parametrized learning rate modulations. Simplified to sidestep the feature/context
        sensitivity issue.
        """

        # computes and cubes similarity value to find activation for each trace in memory
        activation = np.dot(self.memory[:self.encoding_index], probe) / (
            np.sqrt(np.sum(np.square(self.memory[:self.encoding_index]), axis=1)) * np.sqrt(
                np.sum(np.square(probe))))

        # weight activations based on whether probe contains item or contextual features or both
        if np.any(probe[:self.item_count + 1]):
            if np.any(probe[self.item_count + 1:]):
                # both mfc and mcf weightings, see below
                activation *= self.all_weighting[:self.encoding_index]
            else:
                # just mfc weightings - scale by gamma for each experimental trace
                activation *= self.item_weighting[:self.encoding_index]\
        else:
            # just mcf weightings - scale by primacy/attention function based on serial position
            activation *= self.context_weighting[:self.encoding_index]
            
        return activation + 10e-7
    
    def outcome_probabilities(self, activation_cue):
        """
        Outcome probabilities originally depended directly on echo representation corresponding
        to activation cue. Now it depends on the individual trace activations themselves.
        
        
        """
        
        activation = self.activations(activation_cue)
        activation = np.power(activation, self.choice_sensitivity)
        
        self.probabilities = np.zeros((self.presentation_count + 1))
        self.probabilities[0] = min(self.stop_probability_scale * np.exp(
            self.recall_total * self.stop_probability_growth), 1.0 - (self.presentation_count * 10e-7))

        if self.probabilities[0] < 1:
            for already_recalled_item in self.recall[:self.recall_total]:
                echo[int(already_recalled_item)] = 0
        self.probabilities[1:] = (1-self.probabilities[0]) * echo / np.sum(echo)
        
        return self.probabilities

SyntaxError: invalid syntax (Temp/ipykernel_14448/1702804013.py, line 114)

## Likelihood Function

## Demo
First we wanna show that the model (quickly) fits to a single subject's performance on pure list trials.

In [2]:
from compmemlearn.datasets import prepare_murdock1970_data

murd_trials0, murd_events0, murd_length0 = prepare_murdock1970_data('../data/mo1970.txt')

murd_events0.head()

,subject,list,item,input,output,study,recall,repeat,intrusion
0,1,1,1,1,NaN,True,False,0,False
1,1,1,2,2,NaN,True,False,0,False
2,1,1,3,3,NaN,True,False,0,False
3,1,1,4,4,NaN,True,False,0,False
4,1,1,5,5,NaN,True,False,0,False


## Notes

How is CMR different?
- sizes of mcf, context different
- context input is sometimes experience - probably for the delay_context_input and start_context_input
- sampling rule, familiarity stuff
- different definition of weightings

Testing...